In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score,mean_absolute_error,mean_squared_error
from sklearn.metrics import classification_report

In [2]:
classification = pkl.load(open('RFC.pkl','rb'))
regressor = pkl.load(open('reg.pkl','rb'))

In [3]:
df=pd.read_csv('mer.csv')
df

,Origin,Dest,FlightDate,FlightDate.1,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,...,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,ACODE
0,MCO,ORD,2016-12-01,2016-12-01,4,2016,12,1,1404.0,0.0,...,1015,61,67,18,84,82,62,2016-12-01,1400,MCO
1,MCO,MIA,2016-12-01,2016-12-01,4,2016,12,1,1510.0,1.0,...,1015,61,67,18,84,82,62,2016-12-01,1400,MCO
2,MCO,CLT,2016-12-01,2016-12-01,4,2016,12,1,1420.0,1.0,...,1015,61,67,18,84,82,62,2016-12-01,1400,MCO
3,MCO,EWR,2016-12-01,2016-12-01,4,2016,12,1,1518.0,1.0,...,1015,61,67,18,84,82,62,2016-12-01,1400,MCO
4,MCO,JFK,2016-12-01,2016-12-01,4,2016,12,1,1408.0,0.0,...,1015,61,67,18,84,82,62,2016-12-01,1400,MCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864948,DEN,LAS,2017-01-23,2017-01-23,1,2017,1,23,2132.0,0.0,...,1001,0,28,18,39,39,90,2017-01-23,2100,DEN
1864949,DEN,LAX,2017-01-23,2017-01-23,1,2017,1,23,2119.0,0.0,...,1001,0,28,18,39,39,90,2017-01-23,2100,DEN
1864950,DEN,PHX,2017-01-23,2017-01-23,1,2017,1,23,2119.0,1.0,...,1001,0,28,18,39,39,90,2017-01-23,2100,DEN
1864951,DEN,SEA,2017-01-23,2017-01-23,1,2017,1,23,2142.0,0.0,...,1001,0,28,18,39,39,90,2017-01-23,2100,DEN


In [10]:
df.corr()['ArrDel15']

/var/folders/6x/gvhz2ks54ds4gc95v5bfv_7m0000gn/T/ipykernel_5073/1154574463.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['ArrDel15']


Quarter           -0.023270
Year               0.012782
Month             -0.023917
DayofMonth        -0.007646
DepTime            0.163259
DepDel15           0.710134
CRSDepTime         0.128520
DepDelayMinutes    0.546437
OriginAirportID    0.023193
DestAirportID      0.048041
ArrTime            0.055524
CRSArrTime         0.111477
ArrDel15           1.000000
ArrDelayMinutes    0.604086
rounded_time       0.128278
windspeedKmph      0.087623
winddirDegree      0.018735
weatherCode        0.094015
precipMM           0.088868
visibility        -0.020564
pressure          -0.075508
cloudcover         0.039469
DewPointF          0.038007
WindGustKmph       0.080937
tempF              0.037746
WindChillF         0.033876
humidity          -0.003579
time               0.128278
Name: ArrDel15, dtype: float64

In [11]:
df.columns

Index(['Origin', 'Dest', 'FlightDate', 'FlightDate.1', 'Quarter', 'Year',
       'Month', 'DayofMonth', 'DepTime', 'DepDel15', 'CRSDepTime',
       'DepDelayMinutes', 'OriginAirportID', 'DestAirportID', 'ArrTime',
       'CRSArrTime', 'ArrDel15', 'ArrDelayMinutes', 'rounded_time',
       'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
       'tempF', 'WindChillF', 'humidity', 'date', 'time', 'ACODE'],
      dtype='object')

In [4]:
df_1=df.drop(['FlightDate','FlightDate.1','DayofMonth','DepDel15','DepDelayMinutes','OriginAirportID','DestAirportID','time','ArrTime','rounded_time','ACODE','date'],axis=1)

In [5]:
df_1.columns.value_counts()

Origin             1
Dest               1
WindChillF         1
tempF              1
WindGustKmph       1
DewPointF          1
cloudcover         1
pressure           1
visibility         1
precipMM           1
weatherCode        1
winddirDegree      1
windspeedKmph      1
ArrDelayMinutes    1
ArrDel15           1
CRSArrTime         1
CRSDepTime         1
DepTime            1
Month              1
Year               1
Quarter            1
humidity           1
dtype: int64

In [6]:
X=df_1.drop(['ArrDel15','ArrDelayMinutes'],axis=1)
Y=df_1['ArrDel15']


In [7]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
X['Origin']=lb.fit_transform(df_1['Origin'])
X['Dest']=lb.fit_transform(df_1['Dest'])

In [10]:
Y_pred=classification.predict(X)
print("Accuracy =",accuracy_score(Y_pred,Y))
print(classification_report(Y_pred,Y))




Accuracy = 0.9710067760420772
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98   1490035
         1.0       0.91      0.95      0.93    374918

    accuracy                           0.97   1864953
   macro avg       0.95      0.96      0.96   1864953
weighted avg       0.97      0.97      0.97   1864953



In [11]:
df_1=df.drop(['FlightDate','FlightDate.1','OriginAirportID','DestAirportID','time','ArrTime','rounded_time','ACODE','date'],axis=1)

In [12]:
df_1['Y_pred']=Y_pred
df_1.columns

Index(['Origin', 'Dest', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime',
       'DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrDel15',
       'ArrDelayMinutes', 'windspeedKmph', 'winddirDegree', 'weatherCode',
       'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF',
       'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'Y_pred'],
      dtype='object')

In [31]:
Y_pred

array([0., 0., 1., ..., 1., 0., 0.])

In [13]:
reg_df=df_1[df_1['Y_pred']==1]

In [14]:
reg_df.columns.value_counts()

Origin             1
Dest               1
humidity           1
WindChillF         1
tempF              1
WindGustKmph       1
DewPointF          1
cloudcover         1
pressure           1
visibility         1
precipMM           1
weatherCode        1
winddirDegree      1
windspeedKmph      1
ArrDelayMinutes    1
ArrDel15           1
CRSArrTime         1
DepDelayMinutes    1
CRSDepTime         1
DepDel15           1
DepTime            1
DayofMonth         1
Month              1
Year               1
Quarter            1
Y_pred             1
dtype: int64

In [41]:
reg_df.columns

Index(['Origin', 'Dest', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime',
       'DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrDel15',
       'ArrDelayMinutes', 'windspeedKmph', 'winddirDegree', 'weatherCode',
       'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF',
       'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'Y_pred'],
      dtype='object')

In [15]:
pred_df=reg_df.drop(['ArrDel15','Y_pred'],axis=1)

In [16]:
pred_df.columns

Index(['Origin', 'Dest', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime',
       'DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'CRSArrTime',
       'ArrDelayMinutes', 'windspeedKmph', 'winddirDegree', 'weatherCode',
       'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF',
       'WindGustKmph', 'tempF', 'WindChillF', 'humidity'],
      dtype='object')

In [17]:
reg_df

,Origin,Dest,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,...,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,Y_pred
2,MCO,CLT,4,2016,12,1,1420.0,1.0,1355,25.0,...,0.0,10,1015,61,67,18,84,82,62,1.0
3,MCO,EWR,4,2016,12,1,1518.0,1.0,1410,68.0,...,0.0,10,1015,61,67,18,84,82,62,1.0
6,MCO,EWR,4,2016,12,1,1549.0,1.0,1437,72.0,...,0.0,10,1015,61,67,18,84,82,62,1.0
8,MCO,EWR,4,2016,12,1,1434.0,1.0,1410,24.0,...,0.0,10,1015,61,67,18,84,82,62,1.0
12,MCO,EWR,4,2016,12,2,1424.0,0.0,1410,14.0,...,0.0,10,1020,8,50,16,74,72,46,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864931,DEN,LAX,1,2017,1,5,2141.0,1.0,2120,21.0,...,0.0,10,1030,100,1,6,3,-1,91,1.0
1864932,DEN,PHX,1,2017,1,5,2204.0,1.0,2100,64.0,...,0.0,10,1030,100,1,6,3,-1,91,1.0
1864934,DEN,SFO,1,2017,1,5,2205.0,1.0,2140,25.0,...,0.0,10,1030,100,1,6,3,-1,91,1.0
1864935,DEN,LAX,1,2017,1,5,536.0,0.0,540,0.0,...,1.0,7,1027,100,1,14,6,-5,80,1.0


In [18]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
pred_df['Origin']=lb.fit_transform(pred_df['Origin'])
pred_df['Dest']=lb.fit_transform(pred_df['Dest'])

REGRESSION ANALYSIS

PREDICTION AFTER CATEGORY SPLIT

In [19]:
pred_df.columns.value_counts()

Origin             1
Dest               1
WindChillF         1
tempF              1
WindGustKmph       1
DewPointF          1
cloudcover         1
pressure           1
visibility         1
precipMM           1
weatherCode        1
winddirDegree      1
windspeedKmph      1
ArrDelayMinutes    1
CRSArrTime         1
DepDelayMinutes    1
CRSDepTime         1
DepDel15           1
DepTime            1
DayofMonth         1
Month              1
Year               1
Quarter            1
humidity           1
dtype: int64

In [20]:
df_3=pred_df[(pred_df['ArrDelayMinutes']>=15) & (pred_df['ArrDelayMinutes']<100)]
df_4=pred_df[(pred_df['ArrDelayMinutes']>=100) & (pred_df['ArrDelayMinutes']<200)]
df_5=pred_df[(pred_df['ArrDelayMinutes']>=200) & (pred_df['ArrDelayMinutes']<500)]
df_6=pred_df[(pred_df['ArrDelayMinutes']>=500) & (pred_df['ArrDelayMinutes']<1000)]
df_7=pred_df[(pred_df['ArrDelayMinutes']>=1000) & (pred_df['ArrDelayMinutes']<2000)]

In [21]:
type(regressor)

sklearn.ensemble._forest.RandomForestRegressor

In [22]:
def prediction(dataframe):
    X=dataframe.drop(['ArrDelayMinutes'],axis=1)
    Y=dataframe['ArrDelayMinutes']               
    Y_pred=regressor.predict(X)
    print("R2 score",r2_score(Y,Y_pred))
    print("MEAN ABSOUTE ERROR:",mean_absolute_error(Y,Y_pred))
    print("MEAN SQUARE ERROR :",mean_squared_error(Y,Y_pred))
    print("RMSE:",mean_squared_error(Y,Y_pred)**0.5)
    return " "
    

In [23]:
print("Results for Delay ranges 15 to 100 Minutes :\n\n")
print(prediction(df_3),"\n","Total delays:",len(df_3) )
print("\nResults for Delay ranges 100 to 200 Minutes :\n\n")
print(prediction(df_4),"\n","Total delays:",len(df_4) )
print("\nResults for Delay ranges 200 to 500 Minutes :\n\n")
print(prediction(df_5),"\n","Total delays:",len(df_5) )
print("\nResults for Delay ranges 500 to 1000 Minutes :\n\n")
print(prediction(df_6),"\n","Total delays:",len(df_6) )
print("\nResults for Delay ranges 1000 to 2000 Minutes :\n\n")
print(prediction(df_7),"\n","Total delays:",len(df_7) )

Results for Delay ranges 15 to 100 Minutes :


R2 score 0.8060695061294116
MEAN ABSOUTE ERROR: 6.595870958306376
MEAN SQUARE ERROR : 97.04525042852445
RMSE: 9.851154776396747
  
 Total delays: 290596

Results for Delay ranges 100 to 200 Minutes :


R2 score 0.6723524441071811
MEAN ABSOUTE ERROR: 9.00400448641783
MEAN SQUARE ERROR : 239.80224127965337
RMSE: 15.485549434219418
  
 Total delays: 48814

Results for Delay ranges 200 to 500 Minutes :


R2 score 0.9322568829769975
MEAN ABSOUTE ERROR: 9.590284360189573
MEAN SQUARE ERROR : 295.6496250348481
RMSE: 17.19446495343336
  
 Total delays: 14348

Results for Delay ranges 500 to 1000 Minutes :


R2 score 0.9908693435098687
MEAN ABSOUTE ERROR: 8.196252236135958
MEAN SQUARE ERROR : 179.99892906976746
RMSE: 13.416367953726056
  
 Total delays: 1118

Results for Delay ranges 1000 to 2000 Minutes :


R2 score 0.9927943621123495
MEAN ABSOUTE ERROR: 9.647197452229307
MEAN SQUARE ERROR : 193.3332070063697
RMSE: 13.904431200389668
  
 Total dela